In [1]:
import yaml
import xarray as xr

/opt/conda/envs/pyAODN/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/opt/conda/envs/pyAODN/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(scheduler_port=8789)
# cluster

In [3]:
from distributed import LocalCluster
cluster = LocalCluster()
cluster

In [4]:
from distributed import Client
client=Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43661 Dashboard: /proxy/8787/status,Cluster Workers: 2 Cores: 2 Memory: 2.08 GB


In [57]:
client.restart()

Client Scheduler: tcp://146.118.38.63:8789 Dashboard: http://146.118.38.63:8787/status,Cluster Workers: 16 Cores: 16 Memory: 64.00 GB


# Demonstrate the usage of intake-thredds

In [5]:
import intake
cat = intake.open_thredds_cat('http://thredds.aodn.org.au/thredds/catalog/IMOS/SRS/OC/gridded/aqua/P1D/catalog.xml')

ModuleNotFoundError: No module named 'siphon'

Pick out a month - the ['']() syntax is a little ugly, but it works to return the catalogue objects

In [6]:
month=cat['2007']()['02']()
month.url

'http://thredds.aodn.org.au/thredds/catalog/IMOS/SRS/OC/gridded/aqua/P1D/2007/02/catalog.xml'

Print the files available within the month

# Use Dask to walk through the the THREDDS catalog in parallel

Use dask to traverse the catalog in parallel. This involves creating some delayed functions that are chained together

In [1]:
from dask import delayed
from dask.distributed import get_client

@delayed
def obtain_day_urls(url):
    day_urls = []
    cat = intake.open_thredds_cat(url)
    for filename in cat:
        if 'K_490' in filename: # Filter the filenames - just the light attenuation files ('*K_490*')
            day_urls.append(cat[filename].urlpath)
    return day_urls

@delayed
def obtain_month_urls(url):
    month_urls = []
    cat = intake.open_thredds_cat(url)
    for month in cat:
        day_urls=obtain_day_urls(cat[month].url)
        month_urls = month_urls+day_urls
    return month_urls


/opt/conda/envs/pyAODN/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/opt/conda/envs/pyAODN/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


This will return a delayed function call (yet to be evaluated) to request the catalogs for the months of each year

In [59]:
futures=[]
for year in cat:
    if year in ['2014',]:
        print(year)
        futures.append(obtain_month_urls(cat[year].url))
futures

2014


[Delayed('obtain_month_urls-8f6a39f7-61d9-4020-a70a-e828cdec5a62')]

Now send these delayed function calls to the dask scheduler to be computed. First to return the months of each year and then the all the files for each month

In [60]:
from dask import compute
month_urls=compute(futures)[0]
day_urls=compute(month_urls)[0]

In [61]:
# day_urls contains a list of lists for each month, concatentate them all together
all_urls = []
for m in day_urls:
    all_urls=all_urls+m
len(all_urls)

365

Note, now we have the thredds 'dodsC' OpenDAP URLs for each netCDF file

In [24]:
all_urls[0:5]

['http://thredds.aodn.org.au/thredds/dodsC/IMOS/SRS/OC/gridded/aqua/P1D/2013/01/A.P1D.20130101T000000Z.aust.K_490.nc',
 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/SRS/OC/gridded/aqua/P1D/2013/01/A.P1D.20130102T000000Z.aust.K_490.nc',
 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/SRS/OC/gridded/aqua/P1D/2013/01/A.P1D.20130103T000000Z.aust.K_490.nc',
 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/SRS/OC/gridded/aqua/P1D/2013/01/A.P1D.20130104T000000Z.aust.K_490.nc',
 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/SRS/OC/gridded/aqua/P1D/2013/01/A.P1D.20130105T000000Z.aust.K_490.nc']

### Probably good to archive all the URLs somewhere, quicker than hitting the thredds server again

In [25]:
# save/load a yaml file with the URLs for later....
# with open('K490_url.yml', 'w') as outfile:
#     yaml.dump(all_urls, outfile, default_flow_style=False)

In [26]:
# with open('K490_url.yml', 'r') as infile:
#     all_urls=yaml.safe_load(infile)
# all_urls[0:10]

And we can use dask a bit like multiprocessing 'map' and call a delayed function for each URL. Here I am substituting 'fileServer' for dodsC as I want to download the whole file rather than make an OpenDAP request. We can monitor the progress from the dask dashboard link provided above

In [23]:
import urllib.request

@delayed
def download_file(url):
    dest='/scratch/pawsey0106/pbranson/aqua_files/' + url.split('/')[-1]
    urllib.request.urlretrieve(url.replace('dodsC','fileServer'), dest)
    return dest

futures = [download_file(url) for url in all_urls]

files = compute(futures)[0]